# Experimental Analysis of Static Community Detection Using M2V

Running both the Louvain and Leiden algorithms multiple times and recording various statistics for each run can provide valuable insights for our analysis.

## Importing Edge List w/ Weights to NetworkX

NetworkX's read_weighted_edgelist function expects a simple text file with lines of the form <node1> <node2> <weight>, without headers. Since our data is in CSV format, you'll need to use Pandas (or another method) to load the CSV and adjust it to become readable.

In [1]:
import pandas as pd

edge_list_df = pd.read_csv('New Input/M2V_edge_list_with_similarity.csv')

print(edge_list_df)

        source  target    weight
0          u_2    a_51  0.382745
1          u_2    a_52  0.310352
2          u_2    a_53  0.336436
3          u_2    a_54  0.299376
4          u_2    a_55  0.266599
...        ...     ...       ...
160877  u_2099  u_1801  0.621778
160878  u_2099  u_2006  0.625820
160879  u_2099  u_2016  0.535882
160880  u_2100   u_586  0.815714
160881  u_2100   u_607  0.785083

[160882 rows x 3 columns]


In [2]:
negative_weights = edge_list_df[edge_list_df['weight'] < 0]
print(f"Number of edges with negative weights: {len(negative_weights)}")

Number of edges with negative weights: 11


Since Louvain is not made to consider negative edge weights, we will rescale the weights such that instead of [-1, 1] being the range, it is now [0, 1], where 0 now represents perfect dissimilarity, 0.5 represents orthogonality, and 1 represents perfect similarity.

In [3]:
edge_list_df['weight'] = (edge_list_df['weight'] + 1) / 2

print(edge_list_df)

        source  target    weight
0          u_2    a_51  0.691372
1          u_2    a_52  0.655176
2          u_2    a_53  0.668218
3          u_2    a_54  0.649688
4          u_2    a_55  0.633300
...        ...     ...       ...
160877  u_2099  u_1801  0.810889
160878  u_2099  u_2006  0.812910
160879  u_2099  u_2016  0.767941
160880  u_2100   u_586  0.907857
160881  u_2100   u_607  0.892541

[160882 rows x 3 columns]


Before we continue with the creation of a graph, NetworkX specifies that an undirected, weighted graph must not have self-loop, parallel edges (A->B, B->A), or duplicate edges.

In [4]:
duplicate_edges = edge_list_df.duplicated(subset=['source', 'target'], keep=False)
print(f"Number of duplicate edges: {duplicate_edges.sum()}")

self_loops = edge_list_df[edge_list_df['source'] == edge_list_df['target']]
print(f"Number of self-loops: {len(self_loops)}")

print(edge_list_df.isnull().sum())

Number of duplicate edges: 0
Number of self-loops: 0
source    0
target    0
weight    0
dtype: int64


In [5]:
# Find duplicate edges (ignoring the weight column)
duplicate_edges = edge_list_df.duplicated(subset=['source', 'target'], keep=False)

# Filter to get only the duplicate edges
parallel_edges_df = edge_list_df[duplicate_edges]

# Sort to better visualize parallel edges
parallel_edges_sorted = parallel_edges_df.sort_values(by=['source', 'target'])

print(parallel_edges_sorted)

Empty DataFrame
Columns: [source, target, weight]
Index: []


## Creating Undirected Weighted Graph

We iterate over the edge list DataFrame rows to add edges along with their weights to a new NetworkX graph.


In [6]:
import networkx as nx

def get_graph_info(graph):
    print("Number of nodes:", graph.number_of_nodes())
    print("Number of edges:", graph.number_of_edges())
    
    # Checking the graph type to provide appropriate information
    if isinstance(graph, nx.DiGraph):
        print("Graph is Directed")
    else:
        print("Graph is Undirected")


In [7]:
# Initialize a new graph
G = nx.MultiGraph()

# Add edges and weights
for index, row in edge_list_df.iterrows():
    source = row['source']
    target = row['target']
    weight = row['weight']
    
    # Add the edge with weight
    G.add_edge(source, target, weight=weight)

In [8]:
get_graph_info(G)

Number of nodes: 21518
Number of edges: 160882
Graph is Undirected


## Running Louvain Using CDLIB

CDlib (Community Discovery Library) is designed for community detection and analysis, providing easy access to various algorithms, including Louvain and Leiden, and tools for evaluating and visualizing the results.

In [9]:
from cdlib import algorithms
import time
import numpy as np

# Prepare the data structure for results
results = {
    "Louvain": {"modularity": [], "communities": [], "time": []},
    "Leiden": {"modularity": [], "communities": [], "time": []}
}

# Execute each algorithm 10 times
for _ in range(10):
    # Louvain
    start_time = time.time()
    communities_louvain = algorithms.louvain(G, weight='weight')
    elapsed_time = time.time() - start_time
    results["Louvain"]["modularity"].append(communities_louvain.newman_girvan_modularity().score)
    results["Louvain"]["communities"].append(len(communities_louvain.communities))
    results["Louvain"]["time"].append(elapsed_time)
    
    # Leiden
    start_time = time.time()
    communities_leiden = algorithms.leiden(G, weights='weight')
    elapsed_time = time.time() - start_time
    results["Leiden"]["modularity"].append(communities_leiden.newman_girvan_modularity().score)
    results["Leiden"]["communities"].append(len(communities_leiden.communities))
    results["Leiden"]["time"].append(elapsed_time)

# Calculate averages
for method in results:
    results[method]["avg_modularity"] = np.mean(results[method]["modularity"])
    results[method]["avg_time"] = np.mean(results[method]["time"])
    results[method]["avg_communities"] = np.mean(results[method]["communities"])

Note: to be able to use all crisp methods, you need to install some additional packages:  {'bayanpy', 'wurlitzer', 'infomap', 'graph_tool'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'pyclustering', 'ASLPAw'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'wurlitzer', 'infomap'}


In [10]:
# Display the results
print(results)

{'Louvain': {'modularity': [0.6283189622703926, 0.6239206887763508, 0.6207736401838099, 0.6047938075823897, 0.6194606737894817, 0.6263972528573543, 0.6270822986517753, 0.6090804819676654, 0.607820917594113, 0.6281472523656049], 'communities': [526, 524, 569, 602, 530, 559, 504, 576, 611, 533], 'time': [9.93134593963623, 10.062281847000122, 8.868818998336792, 11.677879095077515, 10.732498168945312, 16.685330152511597, 9.61427092552185, 13.594332933425903, 10.162024974822998, 15.877521753311157], 'avg_modularity': 0.6195795976038938, 'avg_time': 11.720630478858947, 'avg_communities': 553.4}, 'Leiden': {'modularity': [0.6483667425717156, 0.6563998740574589, 0.6550958452567309, 0.6576852550101386, 0.6537452446750377, 0.6546388222351064, 0.6508864564381631, 0.6553832415913885, 0.6560937728574486, 0.6564500895912573], 'communities': [15, 18, 16, 17, 15, 16, 17, 16, 19, 14], 'time': [3.166872024536133, 3.2366371154785156, 3.099907159805298, 2.9914469718933105, 3.037999153137207, 3.02690219879

In [11]:
# Convert the results dictionary into a pandas DataFrame
# First, prepare the data in a structured form
data = {
    "Algorithm": [],
    "Run": [],
    "Modularity": [],
    "Communities": [],
    "Time (s)": []
}

# Populate the structured data from the results
for algo in results:
    for run in range(10):  # Assuming 10 runs as previously set
        data["Algorithm"].append(algo)
        data["Run"].append(run + 1)  # Run number (1-10)
        data["Modularity"].append(results[algo]["modularity"][run])
        data["Communities"].append(results[algo]["communities"][run])
        data["Time (s)"].append(results[algo]["time"][run])

# Creating the DataFrame
results_df = pd.DataFrame(data)

# Display the DataFrame for visual inspection
print(results_df)

# Additionally, creating a summary DataFrame for averages
summary_data = {
    "Algorithm": ["Louvain", "Leiden"],
    "Avg. Modularity": [results["Louvain"]["avg_modularity"], results["Leiden"]["avg_modularity"]],
    "Avg. Communities": [results["Louvain"]["avg_communities"], results["Leiden"]["avg_communities"]],
    "Avg. Time (s)": [results["Louvain"]["avg_time"], results["Leiden"]["avg_time"]]
}

summary_df = pd.DataFrame(summary_data)

# Display the summary DataFrame
print(summary_df)

   Algorithm  Run  Modularity  Communities   Time (s)
0    Louvain    1    0.628319          526   9.931346
1    Louvain    2    0.623921          524  10.062282
2    Louvain    3    0.620774          569   8.868819
3    Louvain    4    0.604794          602  11.677879
4    Louvain    5    0.619461          530  10.732498
5    Louvain    6    0.626397          559  16.685330
6    Louvain    7    0.627082          504   9.614271
7    Louvain    8    0.609080          576  13.594333
8    Louvain    9    0.607821          611  10.162025
9    Louvain   10    0.628147          533  15.877522
10    Leiden    1    0.648367           15   3.166872
11    Leiden    2    0.656400           18   3.236637
12    Leiden    3    0.655096           16   3.099907
13    Leiden    4    0.657685           17   2.991447
14    Leiden    5    0.653745           15   3.037999
15    Leiden    6    0.654639           16   3.026902
16    Leiden    7    0.650886           17   3.130058
17    Leiden    8    0.65538

In [12]:
# Save the DataFrame to a CSV file
csv_filename = 'M2V_CD_Experiment_Values.csv'
results_df.to_csv(csv_filename, index=False)